<a href="https://colab.research.google.com/github/idontcalculate/kaggletweets/blob/main/Solution_KaggleTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
!unzip /content/drive/MyDrive/datasets/jigsaw-toxic-comment-classification-challenge.zip

Archive:  /content/drive/MyDrive/datasets/jigsaw-toxic-comment-classification-challenge.zip
  inflating: sample_submission.csv.zip  
  inflating: test.csv.zip            
  inflating: test_labels.csv.zip     
  inflating: train.csv.zip           


In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-06-10 11:58:16--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-06-10 11:58:16 (98.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
!unzip /content/test_labels.csv.zip

!unzip /content/train.csv.zip

!unzip /content/test.csv.zip

import pandas as pd 
train_df = pd.read_csv('/content/train.csv.zip')
test_df  = pd.read_csv('/content/test.csv.zip')

test_labels = pd.read_csv('/content/test_labels.csv.zip')

len(train_df), len(test_df)

# How many samples total?
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

def create_text_label(dataframe):
  texts = dataframe.comment_text.to_numpy()
  labels = dataframe.drop(columns =['id','comment_text']).to_numpy()
  return texts,labels

x_train,y_train = create_text_label(train_df)

x_test,y_test = create_text_label(test_df)

y_test = test_labels.drop('id',axis = 1).to_numpy()

x_train[:10],y_train[:10]

x_test[:10],y_test[:10]

# y_train.head()

len(x_train)

# len(train_data_text[0])

import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long should the output sequence of tokens be?
                                    pad_to_max_tokens=True)

"""For max_tokens (the number of words in the vocabulary), multiples of 10,000 (10,000, 20,000, 30,000) or the exact number of unique words in your text (e.g. 32,179) are common values.

For our use case, we'll use 10,000.

And for the output_sequence_length we'll use the average number of tokens per Tweet in the training set. But first, we'll need to find it.
"""

# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

"""putting dataframe into data_sentences, and then making and array, before vectorizing. """

text_vectorizer.adapt(x_train)
# x_test = text_vectorizer.adapt(np.array(x_test))

"""text_vectorizer.adapt(y_train)"""

text_vectorizer(x_train[:1])

sample_scentence = 'this is the sample scentence'
text_vectorizer([sample_scentence])

text_vocab = text_vectorizer.get_vocabulary()

text_vectorizer(x_test)

# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

"""activating embedding layer"""

from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length) # how long is each input

embedding

# Get a random sentence from training set
import random
random_sentence = random.choice(x_train)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

#single token's embedding
sample_embed[0][0]

"""creating the basseline model"""

x_train.shape, y_train.shape

#SAVE DIR AND CALLBACKS

from helper_functions import create_tensorboard_callback

#create dir to save tensorboard logs
SAVE_DIR = "model_logs"

#print(type(SAVE_DIR))

# Create LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(6, activation="softmax")(x)
model_1 = tf.keras.Model(inputs, outputs, name="LTSM_kaggle")

# Compile model
model_1.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# F model
model_1_history = model_1.fit(x_train,
                              y_train,
                              epochs=5,
                              validation_data=(x_test, y_test),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,  "LSTM_kaggle")])
                              
                              


Archive:  /content/test_labels.csv.zip
replace test_labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  /content/train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: nn
Archive:  /content/test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Total training samples: 159571
Total test samples: 153164
Total samples: 312735
Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'to', 'of']
Bottom 5 least common words: ['domination', 'divorced', 'distorting', 'direktor', 'dildo']
Original text:
shut up man! 

SHUT UP I DID NOT DO ANYTHING Click Here for what I Wrote To You A Few Months Ago.      

Embedded version:
(None, 15, 128)
(None, 64)
Saving TensorBoard log files to: model_logs/LSTM_kaggle/20210610-120635
Epoch 1/5
4987/4987 [==============================] - 140s 27ms/step - loss: 0.3571 - accuracy: 0.0203 - val_loss: -6.8089 - val_accuracy: 0.1393
Epoch 2/5
4987/4987 [==============================] - 113s 23ms/ste

In [ ]:
!pip install tensorboard

In [ ]:
!tensorboard dev upload --logdir ./model_logs/LTSM_kaggle --one_shot


2021-06-10 12:16:53.722952: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./model_logs/LTSM_kaggle

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoa

In [ ]:

SAVE_DIR


'model_logs'

In [ ]:

import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long should the output sequence of tokens be?
                                    pad_to_max_tokens=True)

# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length,
                                    ngrams=2)

#crate model 2 for kaggle

# Create another LTSM 
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(6, activation="softmax")(x)
model_2 = tf.keras.Model(inputs, outputs, name="LTSM_kaggle_2")

# Compile model
model_2.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

#fit the model
model_2_history = model_2.fit(x_train,
                              y_train,
                              epochs=10,
                              validation_data=(x_test, y_test),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       "LTSM_kaggle_2")])

#check the results
model_2.evaluate(x_test, y_test)

#predictions
model_2_pred_probs = model_2.predict(x_test)
model_2_pred_probs.shape

model_2_pred_probs[:10]

# Turn prediction probabilities into single-dimension tensor of floats
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs)) # squeeze removes single dimensions
model_2_preds[:50]

"""results should be in panda dataframe in order to submit. """



(None, 15, 128)
(None, 64)
Saving TensorBoard log files to: model_logs/LTSM_kaggle_2/20210610-121819
Epoch 1/10
4987/4987 [==============================] - 118s 23ms/step - loss: 0.3713 - accuracy: 0.8088 - val_loss: -6.3074 - val_accuracy: 0.9990
Epoch 2/10
4987/4987 [==============================] - 114s 23ms/step - loss: 0.3562 - accuracy: 0.9942 - val_loss: -6.3482 - val_accuracy: 0.9990
Epoch 3/10
4987/4987 [==============================] - 134s 27ms/step - loss: 0.3469 - accuracy: 0.9942 - val_loss: -6.3957 - val_accuracy: 0.9990
Epoch 4/10
4987/4987 [==============================] - 114s 23ms/step - loss: 0.3400 - accuracy: 0.9942 - val_loss: -6.4292 - val_accuracy: 0.9990
Epoch 5/10
4987/4987 [==============================] - 133s 27ms/step - loss: 0.3348 - accuracy: 0.9942 - val_loss: -6.5007 - val_accuracy: 0.9990
Epoch 6/10
4987/4987 [==============================] - 136s 27ms/step - loss: 0.3307 - accuracy: 0.9942 - val_loss: -6.5048 - val_accuracy: 0.9990
Epoch 7/10


'results should be in panda dataframe in order to submit. '

In [ ]:
model_2.save('/content/drive/MyDrive/kaggle_models/final_model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/kaggle_models/final_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/kaggle_models/final_model/assets


In [ ]:
test_labels

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,-1,-1,-1,-1,-1,-1
153160,fffd7a9a6eb32c16,-1,-1,-1,-1,-1,-1
153161,fffda9e8d6fafa9e,-1,-1,-1,-1,-1,-1
153162,fffe8f1340a79fc2,-1,-1,-1,-1,-1,-1


In [ ]:
model_2_pred_probs[:10]

array([[0.3147834 , 0.09744395, 0.21387893, 0.07872009, 0.2040349 ,
        0.09113872],
       [0.3147834 , 0.09744395, 0.21387893, 0.07872009, 0.2040349 ,
        0.09113872],
       [0.3147834 , 0.09744395, 0.21387893, 0.07872009, 0.2040349 ,
        0.09113872],
       [0.3147834 , 0.09744395, 0.21387893, 0.07872009, 0.2040349 ,
        0.09113872],
       [0.3147834 , 0.09744395, 0.21387893, 0.07872009, 0.2040349 ,
        0.09113872],
       [0.3147834 , 0.09744395, 0.21387893, 0.07872009, 0.2040349 ,
        0.09113872],
       [0.3147834 , 0.09744395, 0.21387893, 0.07872009, 0.2040349 ,
        0.09113872],
       [0.3147834 , 0.09744395, 0.21387893, 0.07872009, 0.2040349 ,
        0.09113872],
       [0.3147834 , 0.09744395, 0.21387893, 0.07872009, 0.2040349 ,
        0.09113872],
       [0.3147834 , 0.09744395, 0.21387893, 0.07872009, 0.2040349 ,
        0.09113872]], dtype=float32)

In [ ]:
out_df = pd.DataFrame(model_2_pred_probs)

In [ ]:
out_df.head()

,0,1,2,3,4,5
0,0.314783,0.097444,0.213879,0.07872,0.204035,0.091139
1,0.314783,0.097444,0.213879,0.07872,0.204035,0.091139
2,0.314783,0.097444,0.213879,0.07872,0.204035,0.091139
3,0.314783,0.097444,0.213879,0.07872,0.204035,0.091139
4,0.314783,0.097444,0.213879,0.07872,0.204035,0.091139


In [ ]:
test_labels.columns

Index(['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [ ]:
out_df.columns = [ 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate']

In [ ]:
out_df.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.314783,0.097444,0.213879,0.07872,0.204035,0.091139
1,0.314783,0.097444,0.213879,0.07872,0.204035,0.091139
2,0.314783,0.097444,0.213879,0.07872,0.204035,0.091139
3,0.314783,0.097444,0.213879,0.07872,0.204035,0.091139
4,0.314783,0.097444,0.213879,0.07872,0.204035,0.091139


In [ ]:
test_labels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [ ]:
out_df.insert(0,'id',test_labels['id'])

In [ ]:
out_df

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.314783,0.097444,0.213879,0.078720,0.204035,0.091139
1,0000247867823ef7,0.314783,0.097444,0.213879,0.078720,0.204035,0.091139
2,00013b17ad220c46,0.314783,0.097444,0.213879,0.078720,0.204035,0.091139
3,00017563c3f7919a,0.314783,0.097444,0.213879,0.078720,0.204035,0.091139
4,00017695ad8997eb,0.314783,0.097444,0.213879,0.078720,0.204035,0.091139
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,0.314829,0.097458,0.213806,0.078712,0.204065,0.091130
153160,fffd7a9a6eb32c16,0.314829,0.097458,0.213806,0.078712,0.204065,0.091130
153161,fffda9e8d6fafa9e,0.314829,0.097458,0.213806,0.078712,0.204065,0.091130
153162,fffe8f1340a79fc2,0.314829,0.097458,0.213806,0.078712,0.204065,0.091130


In [ ]:
out_df.to_csv('predictions.csv',index = False)

In [ ]:
out_df.to_csv('/content/drive/MyDrive/datasets/predictions.csv',index = False)

In [ ]:
pd.read_csv('/content/drive/MyDrive/datasets/predictions.csv')

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.314783,0.097444,0.213879,0.078720,0.204035,0.091139
1,0000247867823ef7,0.314783,0.097444,0.213879,0.078720,0.204035,0.091139
2,00013b17ad220c46,0.314783,0.097444,0.213879,0.078720,0.204035,0.091139
3,00017563c3f7919a,0.314783,0.097444,0.213879,0.078720,0.204035,0.091139
4,00017695ad8997eb,0.314783,0.097444,0.213879,0.078720,0.204035,0.091139
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,0.314829,0.097458,0.213806,0.078712,0.204065,0.091130
153160,fffd7a9a6eb32c16,0.314829,0.097458,0.213806,0.078712,0.204065,0.091130
153161,fffda9e8d6fafa9e,0.314829,0.097458,0.213806,0.078712,0.204065,0.091130
153162,fffe8f1340a79fc2,0.314829,0.097458,0.213806,0.078712,0.204065,0.091130
